In [1]:
from client import start_client
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import figure
from main_func import main,real_partial
from jupyterthemes import jtplot

client,w=start_client(5)
client

<Client: 'tcp://127.0.0.1:44075' processes=5 threads=5, memory=33.69 GB>


Client Scheduler: tcp://127.0.0.1:44075 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 5 Memory: 33.69 GB


### Main function
- make_classification parameters: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html?highlight=make_classification#sklearn.datasets.make_classification
- choose e, and chunk size
- run multiple times to produce plots


### Initialize
***

In [2]:
jtplot.style(theme='grade3') #light plot theme

batches=30000 #number of minibatches
batch_size=1 #size of each minibatch
e=0.01 #tolerance for φ(χ)

#compute samples size
training_s=batches*batch_size
n_samples=int(3*training_s/2) # 1/3 of the samples will be used for testing

#parameters for make classification
dataset_params={
    "n_samples":n_samples,
    "n_features":1000,
    "n_informative":2,
    "n_redundant":0,
    "n_repeated":0,
    "n_classes":2,
    "n_clusters_per_class":1,
    "flip_y":0,
    "weights":[0.5,0.5],
    "random_state":17
}
new="yes" #choose if you want to recreate the dataset

### Dataset with no noise y=0
***
#### Balanced
- weights=[0.5,0,5]

In [ ]:
# 3 different e
new="yes"
dataset_params["weights"]=[0.5,0.5]
Bal_acc_e=[]
Bal_rounds_e=[]
for i in [0.01,0.05,0.1]:
    e=i
    E_array,feature_array,Acc,Rounds,Subs=main(client,w,new,dataset_params,500,e)
    new="no"
    Bal_acc_e.append(Acc)
    Bal_rounds_e.append(Rounds)
# 3 different bach_sizes
Bal_acc_b=[]
Bal_rounds_b=[]
for i in [1000,500,150]:
    e=0.01
    batches=i
    E_array,feature_array,Acc,Rounds,Subs=main(client,w,new,dataset_params,batches,e)
    Bal_acc_b.append(Acc)
    Bal_rounds_b.append(Rounds)
    

A new dataset has been created...
coo ([array([ 8.19098293e+01, -5.39423625e+01, -5.94633734e+01,  4.61966053e+00,
       -3.16306636e+01,  1.41500978e+01,  1.71063975e+02, -1.13800372e+02,
       -5.45998188e+01,  2.18317835e+01,  8.92308041e+01, -3.07614162e+01,
        5.69225104e+01,  1.24027548e+01,  1.11310812e+02,  1.45627598e+02,
       -5.93108870e+00, -8.67045004e+01, -4.11354510e+01, -6.02516977e+01,
        6.21800516e+01, -3.56228481e+00, -1.15574479e+01,  1.93841671e+00,
        9.95827837e+01,  1.64796091e+01, -7.41935442e+01, -1.08825097e+02,
        9.13621733e+01,  1.90895903e+01,  1.09892236e+02,  3.26834915e+01,
       -5.09231568e+01,  1.15979379e+02, -4.63445544e+01,  6.32368690e+00,
        1.72813772e+01,  2.57946933e+01,  9.79031625e+01, -1.09284467e+02,
        1.54666936e+01, -1.90120440e+02, -2.31875305e+01,  9.45641382e+01,
        6.93824750e+01,  1.02036529e+02, -1.40860992e+02,  1.69775526e+01,
       -4.65388741e+01,  1.04900822e+02, -1.12089182e+02,  2

coo ([array([ 7.90698353e+01, -1.20356603e+01, -6.70559628e+01, -1.11939736e+01,
       -4.10032788e+01, -6.12526917e+01,  1.70821948e+02, -1.19524698e+02,
        7.70608202e+00, -3.65089982e+01,  1.57752182e+02, -1.07371161e+01,
        1.15465905e+01,  2.71952226e+00,  9.88192212e+01,  1.50087450e+02,
        9.80295270e+01, -5.66413860e+01, -4.54495313e+01, -1.12239141e+02,
        1.01017031e+02, -1.47434270e+00, -9.92586853e+01, -1.95654921e+01,
        7.22006808e+01,  1.71019521e+00, -8.43929627e+01, -1.65670609e+02,
        1.61144100e+01,  8.71910443e+01,  1.01981382e+02,  7.00014145e+01,
       -7.15752999e+00,  9.52413045e+01, -2.45232490e+01, -1.22603202e+01,
       -7.73161104e+00, -2.82681509e+01,  7.36689469e+01, -5.80194917e+01,
       -1.90339934e+01, -1.78551247e+02, -8.14828546e+01,  9.14775568e+01,
        8.79942826e+01,  2.10845126e+02, -1.69117735e+02, -4.50276586e+01,
       -7.89783298e+01,  6.88144942e+01, -4.23312513e+01,  1.94623693e+01,
       -1.16347747e

coo ([array([ 2.15440034e+01, -2.47744752e+01, -7.05285149e+01, -1.24522939e+01,
       -2.64637861e+01, -7.46061367e+01,  1.21228353e+02, -1.65998233e+02,
       -1.70874151e+01, -7.23636050e+01,  1.76260830e+02,  6.89007670e+00,
       -7.64116148e+00,  1.04053110e+01,  1.69498263e+02,  1.25495363e+02,
        9.11945290e+01, -9.21781232e+01, -7.59082276e+01, -1.24712752e+02,
        8.02342774e+01, -4.26569469e+00, -6.20968549e+01, -4.08434025e+01,
        1.03722837e+02,  3.83066687e+01, -8.37380635e+01, -1.70822509e+02,
        2.41773713e+01,  1.08055432e+02,  1.00081489e+02,  1.05756807e+01,
        1.51306537e+01,  4.49841415e+01,  1.83106304e+01,  1.33938609e+01,
       -1.80244973e+01, -1.59200618e+01,  7.09765662e+01, -3.39322534e+01,
       -5.17364135e+01, -1.87649953e+02, -6.98411588e+01,  8.77612609e+01,
        8.58705879e+01,  1.80958183e+02, -1.75748111e+02, -1.41054243e+01,
       -7.67487800e+01,  2.60282218e+01, -5.27860636e+01,  2.55407902e+01,
       -9.93448950e

coo ([array([ 1.07436239e+01,  2.45721208e+01, -8.63786885e+01, -2.07026605e+00,
       -7.90692860e-01, -6.17273895e+01,  4.29993586e+01, -1.64627572e+02,
        1.50973923e+00, -3.12028694e+01,  1.92704729e+02, -6.70632913e+01,
        2.19676691e+00, -2.80819809e+01,  2.12352499e+02,  1.70393267e+02,
        9.07586915e+01, -1.30068516e+02, -4.33414953e+01, -1.22281650e+02,
        8.82426229e+01,  6.34851299e+01, -3.26363070e+01, -4.18205095e+01,
        8.59382346e+01,  2.94353654e+01, -9.26436448e+01, -1.67704493e+02,
        4.32019214e+01,  8.66869354e+01,  1.37842369e+02,  4.64730443e+00,
       -1.13188420e+01,  8.98854507e+01,  9.20565043e-01,  1.56638499e+01,
        9.77052979e+00,  1.30801432e+01,  5.15427963e+01, -3.99473202e+01,
       -4.31163646e+01, -1.75690318e+02, -8.05597938e+01,  5.62340611e+01,
        1.20447504e+02,  1.90106803e+02, -2.21578308e+02, -3.68660480e+00,
       -7.89114712e+01,  3.96620066e+01, -6.24839629e+01,  6.75073459e+01,
       -1.37529213e

coo ([array([ 2.57937585e+00,  2.27728089e+01, -8.94201135e+01, -5.62681465e+01,
       -1.10146264e+02, -8.29246595e+01,  5.27960182e+00, -1.92540011e+02,
       -2.17924140e+01, -2.79716377e+01,  2.27056442e+02, -6.67084361e+01,
        2.33591677e+01, -2.03684100e+01,  2.13765583e+02,  1.74834196e+02,
        8.58559650e+01, -1.02709984e+02, -6.63129979e+01, -1.57327354e+02,
        8.96890294e+01,  7.87676394e+01, -6.36029789e+01,  4.88157647e+00,
        1.33856367e+02, -3.18364792e+00, -1.00472720e+02, -1.45140092e+02,
        1.04755950e+02,  1.04900387e+02,  1.30434314e+02,  1.13696959e+01,
       -7.94215532e+00,  2.90160795e+01,  2.16783951e+01,  2.33158077e+01,
        2.27356483e+01, -3.17555354e+01,  4.11644501e+01, -1.93463590e+01,
       -2.93364149e+01, -1.51534257e+02, -5.26198132e+01,  2.31370771e+01,
        6.65095957e+01,  1.09145964e+02, -2.18563440e+02,  3.63423065e+01,
       -8.04881441e+01,  5.31204996e+01, -4.88305091e+01,  4.55918831e+01,
       -1.32418700e

...

coo ([array([ 5.71070079e+01, -3.34770145e+01, -3.65375469e+01, -6.67696353e-01,
       -1.85889225e+01, -6.34172224e+01,  1.89356855e+01, -2.41870677e+02,
        8.09191021e+01,  4.86763180e+01,  1.55966624e+02, -3.79012726e+00,
        8.94299046e+01, -7.03286638e+01, -6.75206339e+01,  7.61153937e+01,
       -3.30709357e+00, -4.38342787e+01, -9.52852383e+01, -5.43978135e+01,
        4.23395122e+01, -3.80914332e+01, -6.40818121e+01, -5.35687983e+00,
        1.28628766e+02, -1.45117089e+01, -6.11965604e+01, -6.75570333e+01,
        1.04020098e+02,  6.97899616e+00, -7.80624305e-02, -6.40516340e+01,
        7.70224454e+00,  1.20532287e+02, -1.29835544e+02, -1.38357521e+01,
       -3.97592629e+01,  4.79550160e+01, -9.33979977e+00, -6.64190821e+01,
       -3.54093855e-01, -1.27892408e+02, -1.14187058e+02, -1.10627342e+00,
        9.13265019e+01,  7.25648820e+01, -1.03034475e+02, -1.04057702e+01,
        5.72412875e+01,  9.84278648e+01, -6.76025328e+01, -2.05507194e+01,
       -1.37225728e

coo ([array([-5.81049632e+00, -3.48023902e+01, -4.64311465e+01,  1.09984339e+01,
        5.67075640e+00, -6.44503532e+01,  1.78688222e+01, -1.85619357e+02,
        7.49596980e+01,  5.42554512e+01,  1.07009891e+02, -5.27624236e+01,
        8.40962262e+01, -7.41070280e+01, -6.14485827e+01,  2.57954670e+01,
       -8.43657957e+00, -6.85739697e+01, -6.20695413e+01, -7.32423214e+00,
        4.29614714e+01, -5.30315391e+01, -6.50371853e+01, -2.20327782e+01,
        1.05634362e+02,  9.72022114e+00, -3.71266878e+01, -1.01052418e+02,
        7.85536052e+01, -5.50399382e+01,  1.70229950e+00, -3.40629409e+01,
       -2.87384245e+00,  1.56446244e+02, -8.66715010e+01, -2.77342240e+01,
       -2.82337547e+01, -4.66112552e-01,  2.19162792e+01, -5.86180802e+01,
       -6.89254939e+00, -1.09275464e+02, -1.04889862e+02, -1.92779726e+01,
        1.44440783e+02,  4.27432891e+01, -9.66767587e+01, -1.34595387e+01,
        3.42842627e+01,  6.92406465e+01, -6.02960918e+01,  1.66945295e+01,
       -1.00479157e


End of chunks...
Coordinator: finished ...
Workers: ['finished', 'finished', 'finished', 'finished']
coo ([array([-5.81049632e+00, -3.48023902e+01, -4.64311465e+01,  1.09984339e+01,
        5.67075640e+00, -6.44503532e+01,  1.78688222e+01, -1.85619357e+02,
        7.49596980e+01,  5.42554512e+01,  1.07009891e+02, -5.27624236e+01,
        8.40962262e+01, -7.41070280e+01, -6.14485827e+01,  2.57954670e+01,
       -8.43657957e+00, -6.85739697e+01, -6.20695413e+01, -7.32423214e+00,
        4.29614714e+01, -5.30315391e+01, -6.50371853e+01, -2.20327782e+01,
        1.05634362e+02,  9.72022114e+00, -3.71266878e+01, -1.01052418e+02,
        7.85536052e+01, -5.50399382e+01,  1.70229950e+00, -3.40629409e+01,
       -2.87384245e+00,  1.56446244e+02, -8.66715010e+01, -2.77342240e+01,
       -2.82337547e+01, -4.66112552e-01,  2.19162792e+01, -5.86180802e+01,
       -6.89254939e+00, -1.09275464e+02, -1.04889862e+02, -1.92779726e+01,
        1.44440783e+02,  4.27432891e+01, -9.66767587e+01, -1.345953

coo ([array([ 8.19098293e+01, -5.39423625e+01, -5.94633734e+01,  4.61966053e+00,
       -3.16306636e+01,  1.41500978e+01,  1.71063975e+02, -1.13800372e+02,
       -5.45998188e+01,  2.18317835e+01,  8.92308041e+01, -3.07614162e+01,
        5.69225104e+01,  1.24027548e+01,  1.11310812e+02,  1.45627598e+02,
       -5.93108870e+00, -8.67045004e+01, -4.11354510e+01, -6.02516977e+01,
        6.21800516e+01, -3.56228481e+00, -1.15574479e+01,  1.93841671e+00,
        9.95827837e+01,  1.64796091e+01, -7.41935442e+01, -1.08825097e+02,
        9.13621733e+01,  1.90895903e+01,  1.09892236e+02,  3.26834915e+01,
       -5.09231568e+01,  1.15979379e+02, -4.63445544e+01,  6.32368690e+00,
        1.72813772e+01,  2.57946933e+01,  9.79031625e+01, -1.09284467e+02,
        1.54666936e+01, -1.90120440e+02, -2.31875305e+01,  9.45641382e+01,
        6.93824750e+01,  1.02036529e+02, -1.40860992e+02,  1.69775526e+01,
       -4.65388741e+01,  1.04900822e+02, -1.12089182e+02,  2.76954933e+01,
       -1.01036510e

coo ([array([ 2.57937585e+00,  2.27728089e+01, -8.94201135e+01, -5.62681465e+01,
       -1.10146264e+02, -8.29246595e+01,  5.27960182e+00, -1.92540011e+02,
       -2.17924140e+01, -2.79716377e+01,  2.27056442e+02, -6.67084361e+01,
        2.33591677e+01, -2.03684100e+01,  2.13765583e+02,  1.74834196e+02,
        8.58559650e+01, -1.02709984e+02, -6.63129979e+01, -1.57327354e+02,
        8.96890294e+01,  7.87676394e+01, -6.36029789e+01,  4.88157647e+00,
        1.33856367e+02, -3.18364792e+00, -1.00472720e+02, -1.45140092e+02,
        1.04755950e+02,  1.04900387e+02,  1.30434314e+02,  1.13696959e+01,
       -7.94215532e+00,  2.90160795e+01,  2.16783951e+01,  2.33158077e+01,
        2.27356483e+01, -3.17555354e+01,  4.11644501e+01, -1.93463590e+01,
       -2.93364149e+01, -1.51534257e+02, -5.26198132e+01,  2.31370771e+01,
        6.65095957e+01,  1.09145964e+02, -2.18563440e+02,  3.63423065e+01,
       -8.04881441e+01,  5.31204996e+01, -4.88305091e+01,  4.55918831e+01,
       -1.32418700e

In [ ]:
E_real1,Acc_real1=real_partial(100)
len_real1=np.array([i for i in range(len(Acc_real1))])

#### Unbalanced
***
- weights=[0.9,0.1]

In [ ]:
# 3 different e
new="yes"
dataset_params["weights"]=[0.9,0.1]
UnBal_acc_e=[]
UnBal_rounds_e=[]
for i in [0.01,0.05,0.1]:
    e=i
    E_array,feature_array,Acc,Rounds,Subs=main(client,w,new,dataset_params,500,e)
    new="no"
    UnBal_acc_e.append(Acc)
    UnBal_rounds_e.append(Rounds)
# 3 different bach_sizes
UnBal_acc_b=[]
UnBal_rounds_b=[]
for i in [1000,500,150]:
    e=0.01
    batches=i
    E_array,feature_array,Acc,Rounds,Subs=main(client,w,new,dataset_params,batches,e)
    UnBal_acc_b.append(Acc)
    UnBal_rounds_b.append(Rounds)

#### Real

In [ ]:
E_real2,Acc_real2=real_partial(100)
len_real2=np.array([i for i in range(len(Acc_real2))])

### Plots
***
- No noise

In [ ]:
#1.1.a balanced acc-rounds /e (multiple lines one for each e)
figure(figsize=(15,15)) 
plt.subplot(221)
plt.plot(Bal_rounds_e[0], Bal_acc_e[0],label="e=0.01")
plt.plot(Bal_rounds_e[1], Bal_acc_e[1],label="e=0.05")
plt.plot(Bal_rounds_e[2], Bal_acc_e[2],label="e=0.1")
plt.xlabel("Rounds")
plt.ylabel("Accuracy")
plt.title('1.1.a The behaviour changed with different e \n in a no noise balanced dataset')
plt.legend();

plt.subplot(222)
plt.plot(Bal_rounds_b[0], Bal_acc_b[0],label="baches=10")
plt.plot(Bal_rounds_b[1], Bal_acc_b[1],label="baches=100")
plt.plot(Bal_rounds_b[2], Bal_acc_b[2],label="baches=1000")
plt.xlabel("Rounds")
plt.ylabel("Accuracy")
plt.title('1.1.b The behaviour changed with the number of minibatches \n in a no noise balanced dataset')
plt.legend();

#1.2.a unbalanced acc-rounds (multiple lines one for each e)
plt.subplot(223)
plt.plot(UnBal_rounds_e[0], UnBal_acc_e[0],label="e=0.01")
plt.plot(UnBal_rounds_e[1], UnBal_acc_e[1],label="e=0.05")
plt.plot(UnBal_rounds_e[2], UnBal_acc_e[2],label="e=0.1")
plt.xlabel("Rounds")
plt.ylabel("Accuracy")
plt.title('1.2.a The behaviour changed with different e \n in a no noise Unbalanced dataset')
plt.legend();

#1.2.b unbalanced acc-rounds /baches (multiple lines)
plt.subplot(224)
plt.plot(UnBal_rounds_b[0], UnBal_acc_b[0],label="baches=10")
plt.plot(UnBal_rounds_b[1], UnBal_acc_b[1],label="baches=100")
plt.plot(UnBal_rounds_b[2], UnBal_acc_b[2],label="baches=1000")
plt.xlabel("Rounds")
plt.ylabel("Accuracy")
plt.title('1.2.b The behaviour changed with the number of minibatches \n in a no noise Unbalanced dataset')
plt.show();

In [ ]:
plt.plot(len_real1,Acc_real1,label="real")
plt.plot(len_real2,Acc_real2,label="real")
plt.show();

In [ ]:
new="yes"
e=0.1
E_array,feature_array,Acc,Rounds,Subs=main(client,w,new,dataset_params,100,e)